NOTE: This Notebook was used in Google Colab

In [ ]:
# install packages in colab
!pip install optuna
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x8

In [ ]:
# auto reload modules
%load_ext autoreload
%autoreload 0

In [ ]:
# colab specific for using google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# path to data edit
path_to_data = '/content/drive/MyDrive/civol/iceccme/'
filepath_data = path_to_data + 'sample_labeled_by_gpt_3_runs_FINAL.csv'
modelpath = path_to_data + 'models_gpt/'

In [ ]:
# helper function for bringing data into required structure
def get_dataset_from_df(col_text_a, col_text_b, label_col, df):
    dataset = []
    for index, row in df.iterrows():
        text_a = row[col_text_a]
        text_b = row[col_text_b]
        label = row[label_col]
        dataset.append((text_a, text_b, label))
        dataset.append((text_b, text_a, label))
    return dataset


In [ ]:
# import Modules
import logging
from tqdm.autonotebook import tqdm, trange
from sentence_transformers import CrossEncoder, InputExample
from torch.utils.data import DataLoader
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sklearn.model_selection import train_test_split
import optuna
import pandas as pd
import torch

<ipython-input-6-8dfc5c26fce2>:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


tqdm.notebook.tqdm_notebook

In [ ]:
# check for gpu
device = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# read data
df_base = pd.read_csv(filepath_data, sep=';')
df_base['title_description'] = df_base['title'] + ': ' + df_base['description']

In [ ]:
# set logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
# Dataset
dataset = get_dataset_from_df('goal', 'title_description', 'final_label', df_base)

In [ ]:
# callback for store loss and log 
current_loss = None
def simple_callback(loss, epoch, steps):
    global current_loss
    current_loss = loss
    logging.log(logging.INFO, f"Loss: {loss}, epoch: {epoch}, steps: {steps}")

In [ ]:
def objective(trial):
    try:
        # Log the trial number and model creation for tracking purposes
        logging.log(logging.INFO, f"Trial {trial.number}")
        logging.log(logging.INFO, f"Model created")
        
        # Initialize the CrossEncoder model with the specified pre-trained model, setting it to run on CUDA
        model = CrossEncoder('cross-encoder/stsb-roberta-base', num_labels=1, device='cuda')
        
        # Suggest hyperparameters using Optuna's trial object
        learning_rate = trial.suggest_float("learning_rate", 5e-5, 1e-1, log=True)  # Suggest a learning rate between 5e-5 and 0.1 on a log scale
        batch_size = trial.suggest_int("batch_size", 2, 16)  # Suggest an integer batch size between 2 and 16
        num_epochs = trial.suggest_int("num_epochs", 1, 20)  # Suggest an integer number of epochs between 1 and 20
        warmup_steps = trial.suggest_int("warmup_steps", 0, 1000)  # Suggest an integer number of warmup steps between 0 and 1000
        weight_decay = trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True)  # Suggest a weight decay value between 1e-5 and 0.1 on a log scale
        dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)  # Suggest a dropout rate between 0.1 and 0.5

        # Set the dropout rate of the model
        model.dropout_rate = dropout_rate

        # Log the chosen hyperparameters for debugging and tracking purposes
        logging.log(logging.INFO, f"Learning rate: {learning_rate}, batch size: {batch_size}, num epochs: {num_epochs}, warmup steps: {warmup_steps}, weight decay: {weight_decay}, dropout rate: {dropout_rate}")

        # Split the dataset into training and validation sets
        train_data, val_data = train_test_split(dataset, test_size=0.1, random_state=347657)

        # Create DataLoaders for the training and validation sets
        train_dataloader = DataLoader([InputExample(texts=[t[0], t[1]], label=t[2]) for t in train_data], shuffle=True, batch_size=batch_size)
        valid_dataloader = DataLoader([InputExample(texts=[t[0], t[1]], label=t[2]) for t in val_data], batch_size=batch_size)

        # Prepare validation sentence pairs and labels for the evaluator
        val_sentence_pairs = [[t[0], t[1]] for t in val_data]
        labels = [t[2] for t in val_data]

        # Initialize the evaluator for binary classification
        evaluator = CEBinaryClassificationEvaluator(sentence_pairs=val_sentence_pairs, labels=labels, name='goal_eval', write_csv=True, show_progress_bar=True)

        # Fit the model using the specified hyperparameters and data loaders
        model.fit(train_dataloader=train_dataloader,
                  epochs=num_epochs,
                  warmup_steps=warmup_steps,
                  output_path=modelpath+'model_trial_2_'+str(trial.number),
                  optimizer_params={'lr': learning_rate, 'weight_decay': weight_decay},
                  show_progress_bar=True,
                  evaluator=evaluator,
                  evaluation_steps=1000,
                  save_best_model=True,
                  callback=simple_callback
                  )

        # Log that the model fitting is complete
        logging.log(logging.INFO, f"Model fitted")

        # Evaluate the model and log the accuracy
        accuracy = evaluator(model, output_path=modelpath+'model_trial_2_'+str(trial.number))
        logging.log(logging.INFO, f"Accuracy: {accuracy}")

        # Report the accuracy to Optuna
        trial.report(accuracy, trial.number)

        # Check and report current loss if available
        global current_loss
        if current_loss:
            trial.report(current_loss, trial.number)
            logging.log(logging.INFO, f"Trial pruned")
            if trial.should_prune():
                raise optuna.TrialPruned()
                
    except Exception as e:
        # Log any exceptions that occur
        logging.log(logging.ERROR, f"Error: {e}")
        raise e
        
    finally:
        # Clean up by deleting the model and clearing the CUDA cache
        if model:
            del model
            if device == 'cuda':
                torch.cuda.empty_cache()
            elif device == 'mps':
                 torch.mps.empty_cache()

    # Return the accuracy if it was evaluated
    if accuracy:
        return accuracy


In [ ]:
# path to database
storage_path = 'sqlite:///'+modelpath+'db_max_gpt.sqlite3'

In [ ]:
# create and run study
study = optuna.create_study(storage=storage_path, study_name="gpt_data_hpo_max_dc_d", load_if_exists=True, direction="maximize")
study.optimize(objective, n_trials=12)

[I 2024-06-27 07:35:21,771] Using an existing study with name 'own_data_hpo_max_dc_d' instead of creating a new one.
2024-06-27 07:35:21,809 - INFO - Trial 2
2024-06-27 07:35:21,810 - INFO - Model created


pytorch_model.bin:  25%|##5       | 126M/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

2024-06-27 11:50:03,704 - INFO - CEBinaryClassificationEvaluator: Evaluating the model on goal_eval dataset after epoch 10:


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-06-27 11:50:08,686 - INFO - Accuracy:           97.52	(Threshold: 0.0017)
2024-06-27 11:50:08,687 - INFO - F1:                 98.55	(Threshold: 0.0017)
2024-06-27 11:50:08,690 - INFO - Precision:          97.14
2024-06-27 11:50:08,691 - INFO - Recall:             100.00
2024-06-27 11:50:08,694 - INFO - Average Precision:  99.87

2024-06-27 11:50:08,703 - INFO - Loss: 0.9987445444871992, epoch: 10, steps: -1
2024-06-27 11:50:08,704 - INFO - Save model to /content/drive/MyDrive/civol/iceccme/models_gpt/model_trial_2_13
2024-06-27 11:50:11,051 - INFO - Model fitted
2024-06-27 11:50:11,056 - INFO - CEBinaryClassificationEvaluator: Evaluating the model on goal_eval dataset:


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2024-06-27 11:50:15,237 - INFO - Accuracy:           97.52	(Threshold: 0.0017)
2024-06-27 11:50:15,238 - INFO - F1:                 98.55	(Threshold: 0.0017)
2024-06-27 11:50:15,242 - INFO - Precision:          97.14
2024-06-27 11:50:15,244 - INFO - Recall:             100.00
2024-06-27 11:50:15,245 - INFO - Average Precision:  99.87

2024-06-27 11:50:15,259 - INFO - Accuracy: 0.9987445444871992
/usr/local/lib/python3.10/dist-packages/optuna/trial/_trial.py:493: UserWarning: The reported value is ignored because this `step` 13 is already reported.
  warnings.warn(
2024-06-27 11:50:15,274 - INFO - Trial pruned
[I 2024-06-27 11:50:15,478] Trial 13 finished with value: 0.9987445444871992 and parameters: {'learning_rate': 6.500965872736344e-05, 'batch_size': 11, 'num_epochs': 11, 'warmup_steps': 796, 'weight_decay': 0.0427225987615032, 'dropout_rate': 0.10767080665998979}. Best is trial 11 with value: 0.9999999999999999.
